In [1]:
import pandas as pd


In [2]:
# Load in CSV of all female swimmers
file_path = "./usaa_swim_data/female_swimmers_csv.csv"
df = pd.read_csv(file_path)

print(df.head())



         Name      EVENT    TIME  AGE_AT_TIME_OF_SWIM  \
0  Alex Walsh  50 FR SCY   22.08                   16   
1  Alex Walsh  50 FR SCY  22.18r                   18   
2  Alex Walsh  50 FR SCY   22.24                   17   
3  Alex Walsh  50 FR SCY   22.28                   16   
4  Alex Walsh  50 FR SCY   22.35                   16   

                      TIME_CUT                   MEET_PERFORMED  \
0  2018 Summer Nationals (LCM)         2018 SE Southern Premier   
1  2021 Summer Nationals (LCM)  2020 GA Speedo Southern Premier   
2  2019 Summer Nationals (LCM)  2019 SE Speedo Southern Premier   
3  2018 Summer Nationals (LCM)   2017 Winter Junior Nats - East   
4  2018 Winter Nationals (LCM)   2017 Winter Junior Nats - East   

  REGION_OF_MEET                    TEAM SWIM_DATE   Unnamed: 9  Unnamed: 10  \
0             SE  Nashville Aquatic Club    3/3/18  PROGRESSION          NaN   
1             SE  Nashville Aquatic Club    3/6/20  PROGRESSION          NaN   
2             

In [3]:
# Rename relevant columns for clarity
df.rename(columns={
    "NAME": "Name",
    "EVENT": "Event",
    "TIME": "Time",
    "AGE_AT_TIME_OF_SWIM": "Age_at_time_of_Swim",
    "TIME_CUT": "Time_cut", 
    "SWIM_DATE": "Swim_date",
    "SPECIALTY_1": "Specialty_1",
    "SPECIALTY_2": "Specialty_2",
    "STROKES": "Strokes"
}, inplace=True)

In [4]:
def convert_swim_time(time_str):
    """Convert swim times in MM:SS.ss or SS.ss format to total seconds."""
    time_str = str(time_str).strip().replace("r", "")  # Remove 'r' and extra spaces
    
    if ":" in time_str:  # If the time is in MM:SS.ss format
        minutes, seconds = time_str.split(":")
        return float(minutes) * 60 + float(seconds)
    else:  # If the time is already in SS.ss format
        return float(time_str)

In [5]:
# Clean Columns one by one: 

# Name - N/A
# Event - N/A
# Age at time of swim - NA

# Time Cut - N/A
# Swim Date - Convert Swim_date to datetime format
# If your dates are in "MM/DD/YYYY" format:
df["Swim_date"] = df["Swim_date"].astype(str)
# Convert M/D/YY to proper datetime format
df["Swim_date"] = pd.to_datetime(df["Swim_date"], format="%m/%d/%y", errors="coerce")

# Time - Remove trailing 'r' from Time and convert to float
# Apply conversion to the Time column
df["Time"] = df["Time"].apply(convert_swim_time)

# Print to verify
print(df[["Name", "Event", "Time", "Swim_date"]].head())

# Continued in next code block

         Name      Event   Time  Swim_date
0  Alex Walsh  50 FR SCY  22.08 2018-03-03
1  Alex Walsh  50 FR SCY  22.18 2020-03-06
2  Alex Walsh  50 FR SCY  22.24 2019-03-02
3  Alex Walsh  50 FR SCY  22.28 2017-12-07
4  Alex Walsh  50 FR SCY  22.35 2017-12-07


In [6]:
# Clean Columns one by one - continued 

# Specialty_1: N/A
# Specialty_2: N/A
# Ensure the column exists and is of string type before splitting
df["Strokes"] = df["Strokes"].fillna("").astype(str)

# Split 'Strokes' column into up to 3 separate stroke columns
stroke_split = df["Strokes"].str.split(",", expand=True)

# Assign stroke columns dynamically (handling cases with fewer than 3 strokes)
df["Stroke_1"] = stroke_split[0].str.strip().fillna("None")
df["Stroke_2"] = stroke_split[1].str.strip().fillna("None") if stroke_split.shape[1] > 1 else "None"
df["Stroke_3"] = stroke_split[2].str.strip().fillna("None") if stroke_split.shape[1] > 2 else "None"

# Print sample output to verify
print(df[["Strokes", "Stroke_1", "Stroke_2", "Stroke_3"]].head())

            Strokes Stroke_1 Stroke_2 Stroke_3
0  IM, Breast, Back       IM   Breast     Back
1  IM, Breast, Back       IM   Breast     Back
2  IM, Breast, Back       IM   Breast     Back
3  IM, Breast, Back       IM   Breast     Back
4  IM, Breast, Back       IM   Breast     Back


In [7]:
# Define time cut point system
time_cut_points = {
    "Olympic Trials": 24,
    "Summer Nationals": 19,
    "Winter Nationals": 18,
    "Winter US Open": 15,
    "Summer Juniors": 12,
    "Winter Juniors": 10,
    "Futures": 8,
    "AAAA": 7,
    "AAA": 6,
    "AA": 5,
    "A": 4,
    "BB": 3,
    "B": 2,
    "Slower Than B": 1
}

# Assign points based on partial match in "Time_cut"
def assign_time_cut_points(time_cut_value):
    if pd.isna(time_cut_value):
        return 0
    for cut_name, points in time_cut_points.items():
        if cut_name in time_cut_value:
            return points
    return 0

# Apply function to Time_cut column
df["Time_cut_points"] = df["Time_cut"].apply(assign_time_cut_points)

# Print sample output
print(df[["Time_cut", "Time_cut_points"]].head())

                      Time_cut  Time_cut_points
0  2018 Summer Nationals (LCM)               19
1  2021 Summer Nationals (LCM)               19
2  2019 Summer Nationals (LCM)               19
3  2018 Summer Nationals (LCM)               19
4  2018 Winter Nationals (LCM)               18


In [8]:
columns_to_keep = ["Name", "Event", "Time", "Age_at_time_of_Swim", "Time_cut", "Time_cut_points", "Swim_date", "Specialty_1", "Specialty_2", "Stroke_1", "Stroke_2", "Stroke_3"]

# Select only relevant columns for a cleaned data frame
df_cleaned = df[columns_to_keep]



print(df_cleaned.head())


         Name      Event   Time  Age_at_time_of_Swim  \
0  Alex Walsh  50 FR SCY  22.08                   16   
1  Alex Walsh  50 FR SCY  22.18                   18   
2  Alex Walsh  50 FR SCY  22.24                   17   
3  Alex Walsh  50 FR SCY  22.28                   16   
4  Alex Walsh  50 FR SCY  22.35                   16   

                      Time_cut  Time_cut_points  Swim_date Specialty_1  \
0  2018 Summer Nationals (LCM)               19 2018-03-03      200 IM   
1  2021 Summer Nationals (LCM)               19 2020-03-06      200 IM   
2  2019 Summer Nationals (LCM)               19 2019-03-02      200 IM   
3  2018 Summer Nationals (LCM)               19 2017-12-07      200 IM   
4  2018 Winter Nationals (LCM)               18 2017-12-07      200 IM   

  Specialty_2 Stroke_1 Stroke_2 Stroke_3  
0  200 Breast       IM   Breast     Back  
1  200 Breast       IM   Breast     Back  
2  200 Breast       IM   Breast     Back  
3  200 Breast       IM   Breast     Back  
4  

In [9]:

# Save cleaned data
cleaned_file_path = "./usaa_swim_data/swimmers_cleaned.csv"
df_cleaned.to_csv(cleaned_file_path, index=False)

# Print the first few rows of cleaned data
print(df_cleaned.head())

         Name      Event   Time  Age_at_time_of_Swim  \
0  Alex Walsh  50 FR SCY  22.08                   16   
1  Alex Walsh  50 FR SCY  22.18                   18   
2  Alex Walsh  50 FR SCY  22.24                   17   
3  Alex Walsh  50 FR SCY  22.28                   16   
4  Alex Walsh  50 FR SCY  22.35                   16   

                      Time_cut  Time_cut_points  Swim_date Specialty_1  \
0  2018 Summer Nationals (LCM)               19 2018-03-03      200 IM   
1  2021 Summer Nationals (LCM)               19 2020-03-06      200 IM   
2  2019 Summer Nationals (LCM)               19 2019-03-02      200 IM   
3  2018 Summer Nationals (LCM)               19 2017-12-07      200 IM   
4  2018 Winter Nationals (LCM)               18 2017-12-07      200 IM   

  Specialty_2 Stroke_1 Stroke_2 Stroke_3  
0  200 Breast       IM   Breast     Back  
1  200 Breast       IM   Breast     Back  
2  200 Breast       IM   Breast     Back  
3  200 Breast       IM   Breast     Back  
4  